In [1]:
import sqlite3
import utils
import pandas as pd
from sklearn.model_selection import train_test_split
from numba import cuda

from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import numpy as np
import torch

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

from keras.layers import Conv1D, GlobalMaxPooling1D,LSTM

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
import keras.backend as K

#dtype='float16'
#K.set_floatx(dtype)

# default is 1e-7 which is too small for float16.  Without adjusting the epsilon, we will get NaN predictions because of divide by zero problems
#K.set_epsilon(1e-3) 

In [3]:
import tensorflow as tf

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11940406395894384696
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4557766656
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13214430501901989942
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [5]:
def free_gpu_mem():
    cuda.select_device(0)
    cuda.close()

In [6]:
def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

In [7]:
word_vec_size=200

In [8]:
def w2v_per_token_wiki(word_vec_size):
    conn=sqlite3.connect('songs.db')
    cur=conn.cursor()
    example_text=cur.execute("select Lyrics from song where Lyrics is not null").fetchall()
    labels=[i[0] for i in cur.execute("select Economic from song where Lyrics is not null").fetchall()]
    songs=[]
    for i in example_text:
        songs.append(utils.tokenize_word(utils.clean_text(i[0])))

    embeddings_index,songs=utils.get_embedding_dict(songs,'glove-twitter-200')
    print(len(embeddings_index.keys()))
    max_words=len(embeddings_index.keys())
    X,y=np.array(songs),np.array(labels)

    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)

    length = []
    for x in X:
        length.append(len(x))

    X = pad_sequences(sequences, maxlen=max(length)+2)

    embedding_matrix = np.zeros((max_words, word_vec_size))
    for word, i in tokenizer.word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.5)
    
    
    return x_train,x_test,y_train,y_test,embedding_matrix,max(length)+2,max_words

In [9]:
x_train,x_test,y_train,y_test,embedding_matrix,max_sent_len,max_words=w2v_per_token_wiki(word_vec_size)

21576


In [10]:
def dummy(x_train,x_test,y_train,y_test):
    dummy = DummyClassifier()
    dummy.fit(x_train, y_train)
    y_pred=dummy.predict(x_test)
    return f1_score(y_test,y_pred)

In [11]:
print(dummy(x_train,x_test,y_train,y_test))

0.45317840054682157


In [12]:
def dnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model_ptw2v = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model_ptw2v.add(e)
    model_ptw2v.add(Flatten())
    model_ptw2v.add(Dense(128, activation='relu'))
    model_ptw2v.add(Dropout(0.25))
    model_ptw2v.add(Dense(1, activation='sigmoid'))
    model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_ptw2v.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=64, verbose=2)
    y_pred = [int(round(i[0])) for i in model_ptw2v.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [13]:
print(dnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 6s - loss: 0.7637 - acc: 0.5792 - val_loss: 0.5757 - val_acc: 0.6397
Epoch 2/100
 - 3s - loss: 0.3950 - acc: 0.8269 - val_loss: 0.5629 - val_acc: 0.6556
Epoch 3/100
 - 3s - loss: 0.2243 - acc: 0.9312 - val_loss: 0.6305 - val_acc: 0.6541
Epoch 4/100
 - 3s - loss: 0.1372 - acc: 0.9690 - val_loss: 0.7035 - val_acc: 0.6526
Epoch 5/100
 - 3s - loss: 0.0911 - acc: 0.9796 - val_loss: 0.7454 - val_acc: 0.6541
Epoch 6/100
 - 3s - loss: 0.0625 - acc: 0.9874 - val_loss: 0.8499 - val_acc: 0.6541
Epoch 7/100
 - 3s - loss: 0.0640 - acc: 0.9877 - val_loss: 0.8898 - val_acc: 0.6550
Epoch 8/100
 - 3s - loss: 0.0562 - acc: 0.9886 - val_loss: 0.9075 - val_acc: 0.6493
Epoch 9/100
 - 3s - loss: 0.0484 - acc: 0.987

Epoch 86/100
 - 3s - loss: 0.0509 - acc: 0.9907 - val_loss: 2.3378 - val_acc: 0.6614
Epoch 87/100
 - 3s - loss: 0.0531 - acc: 0.9898 - val_loss: 2.0911 - val_acc: 0.6556
Epoch 88/100
 - 3s - loss: 0.0534 - acc: 0.9904 - val_loss: 2.1702 - val_acc: 0.6493
Epoch 89/100
 - 3s - loss: 0.0541 - acc: 0.9901 - val_loss: 1.9718 - val_acc: 0.6626
Epoch 90/100
 - 3s - loss: 0.0541 - acc: 0.9904 - val_loss: 2.2475 - val_acc: 0.6656
Epoch 91/100
 - 3s - loss: 0.0517 - acc: 0.9919 - val_loss: 2.0617 - val_acc: 0.6569
Epoch 92/100
 - 3s - loss: 0.0515 - acc: 0.9910 - val_loss: 2.1784 - val_acc: 0.6562
Epoch 93/100
 - 3s - loss: 0.0458 - acc: 0.9913 - val_loss: 2.2278 - val_acc: 0.6493
Epoch 94/100
 - 3s - loss: 0.0559 - acc: 0.9919 - val_loss: 2.1804 - val_acc: 0.6587
Epoch 95/100
 - 3s - loss: 0.0532 - acc: 0.9913 - val_loss: 2.6261 - val_acc: 0.6556
Epoch 96/100
 - 3s - loss: 0.0672 - acc: 0.9904 - val_loss: 2.5217 - val_acc: 0.6454
Epoch 97/100
 - 3s - loss: 0.0919 - acc: 0.9880 - val_loss: 2.579

In [14]:
def dnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model_ptw2v = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model_ptw2v.add(e)
    model_ptw2v.add(Flatten())
    model_ptw2v.add(Dense(128, activation='relu'))
    model_ptw2v.add(Dropout(0.25))
    model_ptw2v.add(Dense(1, activation='sigmoid'))
    model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_ptw2v.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=64, verbose=2)
    y_pred = [int(round(i[0])) for i in model_ptw2v.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [15]:
print(dnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 6s - loss: 0.8654 - acc: 0.5786 - val_loss: 0.5937 - val_acc: 0.6388
Epoch 2/100
 - 4s - loss: 0.4408 - acc: 0.7932 - val_loss: 0.5767 - val_acc: 0.6569
Epoch 3/100
 - 4s - loss: 0.2468 - acc: 0.9191 - val_loss: 0.6235 - val_acc: 0.6602
Epoch 4/100
 - 4s - loss: 0.1177 - acc: 0.9708 - val_loss: 0.7211 - val_acc: 0.6587
Epoch 5/100
 - 4s - loss: 0.0730 - acc: 0.9817 - val_loss: 0.7994 - val_acc: 0.6520
Epoch 6/100
 - 4s - loss: 0.0580 - acc: 0.9868 - val_loss: 0.8592 - val_acc: 0.6523
Epoch 7/100
 - 4s - loss: 0.0571 - acc: 0.9835 - val_loss: 0.8800 - val_acc: 0.6587
Epoch 8/100
 - 4s - loss: 0.0498 - acc: 0.9865 - val_loss: 0.9005 - val_acc: 0.6547
Epoch 9/100
 - 4s - loss: 0.0428 - acc: 0.9862 - val_loss: 0.9235 - val_acc: 0.6517
Epoch 10/100
 - 4s - loss: 0.0439 - acc: 0.9862 - val_loss: 0.9698 - val_acc: 0.6553
Epoch 11/100
 - 4s - loss: 0.0410 - acc: 0.9898 - val_loss: 0.9706 - val_acc: 0.6578
Epoch 12/100
 - 4s - loss:

Epoch 97/100
 - 4s - loss: 0.0524 - acc: 0.9904 - val_loss: 1.9969 - val_acc: 0.6602
Epoch 98/100
 - 4s - loss: 0.0475 - acc: 0.9907 - val_loss: 1.8717 - val_acc: 0.6514
Epoch 99/100
 - 4s - loss: 0.0522 - acc: 0.9916 - val_loss: 2.1150 - val_acc: 0.6593
Epoch 100/100
 - 4s - loss: 0.0514 - acc: 0.9919 - val_loss: 2.1254 - val_acc: 0.6641
0.5528


In [16]:
def cnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model.add(e)
    model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=32, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [17]:
print(cnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 4s - loss: 0.6862 - acc: 0.5630 - val_loss: 0.6542 - val_acc: 0.6184
Epoch 2/100
 - 2s - loss: 0.5912 - acc: 0.6808 - val_loss: 0.6148 - val_acc: 0.6424
Epoch 3/100
 - 2s - loss: 0.4665 - acc: 0.7926 - val_loss: 0.6034 - val_acc: 0.6475
Epoch 4/100
 - 2s - loss: 0.3138 - acc: 0.8837 - val_loss: 0.6315 - val_acc: 0.6445
Epoch 5/100
 - 2s - loss: 0.1949 - acc: 0.9399 - val_loss: 0.7361 - val_acc: 0.6514
Epoch 6/100
 - 2s - loss: 0.1254 - acc: 0.9708 - val_loss: 0.8247 - val_acc: 0.6364
Epoch 7/100
 - 2s - loss: 0.0932 - acc: 0.9823 - val_loss: 0.8647 - val_acc: 0.6481
Epoch 8/100
 - 2s - loss: 0.0755 - acc: 0.9865 - val_loss: 0.9495 - val_acc: 0.6331
Epoch 9/100
 - 2s - loss: 0.0843 - acc: 0.9856 - val_loss: 0.9053 - val_acc: 0.6352
Epoch 10/100
 - 2s - loss: 0.0743 - acc: 0.9841 - val_loss: 0.9933 - val_acc: 0.6406
Epoch 11/100
 - 2s - loss: 0.0702

Epoch 94/100
 - 2s - loss: 0.0642 - acc: 0.9763 - val_loss: 1.2906 - val_acc: 0.6454
Epoch 95/100
 - 2s - loss: 0.0913 - acc: 0.9663 - val_loss: 1.6769 - val_acc: 0.6391
Epoch 96/100
 - 2s - loss: 0.0426 - acc: 0.9841 - val_loss: 1.5070 - val_acc: 0.6454
Epoch 97/100
 - 2s - loss: 0.0246 - acc: 0.9898 - val_loss: 1.9726 - val_acc: 0.6376
Epoch 98/100
 - 2s - loss: 0.0193 - acc: 0.9922 - val_loss: 2.1067 - val_acc: 0.6412
Epoch 99/100
 - 2s - loss: 0.0171 - acc: 0.9919 - val_loss: 2.1406 - val_acc: 0.6457
Epoch 100/100
 - 2s - loss: 0.0166 - acc: 0.9919 - val_loss: 2.1190 - val_acc: 0.6421
0.5813708260105448


In [18]:
def cnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model.add(e)
    model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=32, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [19]:
print(cnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 3s - loss: 0.6932 - acc: 0.5687 - val_loss: 0.6348 - val_acc: 0.6181
Epoch 2/100
 - 2s - loss: 0.5298 - acc: 0.7463 - val_loss: 0.5699 - val_acc: 0.6466
Epoch 3/100
 - 2s - loss: 0.2855 - acc: 0.9095 - val_loss: 0.6047 - val_acc: 0.6523
Epoch 4/100
 - 2s - loss: 0.1171 - acc: 0.9745 - val_loss: 0.7279 - val_acc: 0.6427
Epoch 5/100
 - 2s - loss: 0.0859 - acc: 0.9829 - val_loss: 0.7226 - val_acc: 0.6541
Epoch 6/100
 - 2s - loss: 0.0731 - acc: 0.9868 - val_loss: 0.7320 - val_acc: 0.6541
Epoch 7/100
 - 2s - loss: 0.0616 - acc: 0.9871 - val_loss: 0.7512 - val_acc: 0.6538
Epoch 8/100
 - 2s - loss: 0.0545 - acc: 0.9871 - val_loss: 0.7913 - val_acc: 0.6439
Epoch 9/100
 - 2s - loss: 0.0500 - acc: 0.9871 - val_loss: 0.7350 - val_acc: 0.6559
Epoch 10/100
 - 2s - loss: 0.0477 - acc: 0.9871 - val_loss: 0.7985 - val_acc: 0.6496
Epoch 11/100
 - 2s - loss: 0.0335 - acc: 0.9880 - val_loss: 0.7949 - val_acc: 0.6541
Epoch 12/100
 - 2s - loss:

Epoch 97/100
 - 2s - loss: 0.0211 - acc: 0.9910 - val_loss: 1.9324 - val_acc: 0.6617
Epoch 98/100
 - 2s - loss: 0.0205 - acc: 0.9925 - val_loss: 2.0259 - val_acc: 0.6623
Epoch 99/100
 - 2s - loss: 0.0220 - acc: 0.9913 - val_loss: 1.8072 - val_acc: 0.6596
Epoch 100/100
 - 2s - loss: 0.0209 - acc: 0.9907 - val_loss: 1.6099 - val_acc: 0.6544
0.6130551816958277


In [20]:
def lstm_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model.add(e)
    model.add(LSTM(100))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [21]:
print(lstm_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/10
 - 385s - loss: 0.6857 - acc: 0.5555 - val_loss: 0.6768 - val_acc: 0.5688
Epoch 2/10
 - 382s - loss: 0.6621 - acc: 0.5930 - val_loss: 0.6803 - val_acc: 0.5655
Epoch 3/10
 - 382s - loss: 0.6411 - acc: 0.6300 - val_loss: 0.6591 - val_acc: 0.6010
Epoch 4/10
 - 383s - loss: 0.6073 - acc: 0.6679 - val_loss: 0.6509 - val_acc: 0.6121
Epoch 5/10
 - 383s - loss: 0.5651 - acc: 0.7048 - val_loss: 0.6362 - val_acc: 0.6334
Epoch 6/10
 - 382s - loss: 0.4940 - acc: 0.7511 - val_loss: 0.6749 - val_acc: 0.6382
Epoch 7/10
 - 382s - loss: 0.4259 - acc: 0.7980 - val_loss: 0.6761 - val_acc: 0.6397
Epoch 8/10
 - 383s - loss: 0.3440 - acc: 0.8506 - val_loss: 0.7249 - val_acc: 0.6499
Epoch 9/10
 - 380s - loss: 0.2693 - acc: 0.8945 - val_loss: 0.8290 - val_acc: 0.6538
Epoch 10/10
 - 380s - loss: 0.1863 - acc: 0.9291 - val_loss: 1.0320 - val_acc: 0.6451
0.5973406068871462


In [22]:
def lstm_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model.add(e)
    model.add(LSTM(100))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [23]:
print(lstm_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/10
 - 397s - loss: 0.6826 - acc: 0.5537 - val_loss: 0.6706 - val_acc: 0.5748
Epoch 2/10
 - 397s - loss: 0.6360 - acc: 0.6417 - val_loss: 0.6446 - val_acc: 0.6211
Epoch 3/10
 - 396s - loss: 0.5504 - acc: 0.7244 - val_loss: 0.6574 - val_acc: 0.6349
Epoch 4/10
 - 398s - loss: 0.4269 - acc: 0.8052 - val_loss: 0.6641 - val_acc: 0.6382
Epoch 5/10
 - 397s - loss: 0.3119 - acc: 0.8659 - val_loss: 0.7713 - val_acc: 0.6499
Epoch 6/10
 - 397s - loss: 0.2121 - acc: 0.9137 - val_loss: 0.9191 - val_acc: 0.6581
Epoch 7/10
 - 396s - loss: 0.1397 - acc: 0.9459 - val_loss: 1.2554 - val_acc: 0.6502
Epoch 8/10
 - 399s - loss: 0.0961 - acc: 0.9660 - val_loss: 1.3878 - val_acc: 0.6547
Epoch 9/10
 - 398s - loss: 0.0736 - acc: 0.9723 - val_loss: 1.3185 - val_acc: 0.6547
Epoch 10/10
 - 399s - loss: 0.0566 - acc: 0.9814 - val_loss: 1.4639 - val_acc: 0.6608
0.6282515640434639
